# Programmation par contrainte: évaluation pratique
## 5-SDBD + mastère VALDOM, INSA Toulouse, janvier 2023

## Première partie

On utilise tout au long de ce sujet deux ensembles d’agents H = {h1, . . . hn} et F = {f1, . . . fn}. Chaque agent
hi ∈ H exprime ses préférences de couplage avec un agent de F à travers une liste L(hi) contenant les éléments de F
sans duplication. Pour tout k ∈ [1, n − 1], hi préfère être couplé avec le kème agent dans L(hi) que le k + 1ème agent.
De la même façon, chaque agent fj ∈ F exprime ses préférences de couplage avec un agent de H à travers une liste
L(fj ).

Un tuple (h, f ) est appelé un couple si h ∈ H et f ∈ F . Soit M un ensemble de couples. Un agent a préfère un
agent b à sa situation dans M si a n’appartient à aucun couple dans M ou si a préfère b à son partenaire dans M . Un
couple (h, f ) ∈ M bloque M si h préfère f à sa situation dans M et f préfère h à sa situation dans M .
Un mariage stable est un ensemble de couples M tel que chaque agent est couplé avec un seul agent et M n’admet
aucun couple bloquant. On note qu’une instance de ce problème peut admettre plusieurs solutions.

In [213]:
from config import setup
setup()

from docplex.cp.model import *
from docplex.cp.config import get_default

import numpy as np
%matplotlib inline

1. Proposez un modèle de programmation par contrainte pour trouver un mariage stable. Le modèle peut inclure
des contraintes logiques (par exemple if_then). L’utilisation de ce genre de contraintes est accessible dans la
documentation du solveur. Utilisez la recherche en profondeur par défaut ((’DepthFirst’) sans préciser des
heuristiques de branchements. N’affichez pas les traces d’exécutions internes du solveur.

In [214]:
from random import shuffle

def generate_SM(n):
    
    Lh = []
    Lf = []
    for i in range(n):
        h = list(set(list(range(n))))
        f = list(set(list(range(n))))
        shuffle(h)
        Lh.append(h)
        shuffle(f)
        Lf.append(f)
        
    return Lh, Lf

In [181]:
Lf, Lh = generate_SM(4)

In [215]:
#Données du problème

#Nombre d'agents dans chaque groupe
n = 4

#Matrice de préférences de H (liste de listes)
#La préférence est décroissante : plus un agent a un indice bas, plus la préférence est élevée
Lh_old = [[2,3,1,4], 
      [4,1,3,2],
      [2,4,1,3],
      [3,1,4,2]]
one = np.ones((4,4), dtype = int)
Lh = (Lh_old - one).tolist()

#Matrice de préférences de F
Lf_old = [[2,1,3,4],
      [3,4,1,2],
      [1,3,4,2],
      [2,1,3,4]]
Lf = (Lf_old - one).tolist()

print(Lh, "\n\n", Lf)

[[1, 2, 0, 3], [3, 0, 2, 1], [1, 3, 0, 2], [2, 0, 3, 1]] 

 [[1, 0, 2, 3], [2, 3, 0, 1], [0, 2, 3, 1], [1, 0, 2, 3]]


In [223]:
mdl = CpoModel(name='couples stables')

#Variables
#Représentation des couples : l'agent h_i est en couple avec l'agent f_couple[i]
#Exemple : si couple = [1, 3, 4, 2] alors l'agent h2 est en couple avec f4 
couples = mdl.integer_var_list(n,0,n-1,'c')
h_index = [mdl.integer_var_list(n, 0, n-1, name=f'ih_{i}') for i in range(n)]
f_index = [mdl.integer_var_list(n, 0, n-1, name=f'if_{i}') for i in range(n)]


#Contraintes

for i in range(n):
    for j in range(n):
        #L'index de hi selon les préférences de fj
        mdl.add(h_index[i][j] == Lf[j].index(i))
        #L'index de fi selon les préférences de hj
        mdl.add(f_index[i][j] == Lh[j].index(i))

#Chaque agent ne peut être que dans un seul couple
mdl.add(mdl.all_diff(couples))

#Quand on a le choix entre prendre Lh[agent][i] et Lh[agent][j] où i<j, on prend Lh[agent][i]
#for i in range(n):
#    for j in range(n):
#        mdl.add(mdl.if_then(couples[i] == j, Lh[i][j] <= min([Lh[i][k] for k in range(n) if k != j])))

#Non bloquant 
#(où bloquant : f préfère h et h préfère f, où f et h sont dans 2 couples différents)
for i in range(n):
    for j in range(n):
        h_index_i_j = h_index[i][j]
        f_index_j_i = f_index[j][i]
        h_index_i_c = mdl.element(h_index[i], couples[i])
        f_index_j_c = mdl.element(f_index[j], couples[j])
        #si hi préfère fj à sa situation actuelle (f_ci), alors il faut que fj préfère sa situation actuelle à hi
        mdl.add(mdl.if_then(mdl.less_or_equal(f_index_i_j,h_index_i_c), greater_or_equal(f_index_j_i,f_index_j_c)))


#print(mdl.get_cpo_string())

In [224]:
sol = mdl.solve(trace_log=True)

 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Satisfiability problem - 37 variables, 49 constraints
 ! Presolve             = Off
 ! Workers              = 1
 ! SearchType           = DepthFirst
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 74.0 (before), 74.0 (after)
 !  . Memory usage      : 300.9 kB (before), 300.9 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed            Branch decision
 *                      4  0.01s                  2  = if_0_0@1
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 4
 ! Number of fails        : 0
 ! Total memory usage     : 641.4 kB (599.1 kB CP Optimizer + 42.4 kB Concert)
 ! Time

In [225]:
def print_SM(sol):
    res = "L'ensemble M={"
    for i in range(len(couples)):
        res = res + "(h_" + str(i) + ",f_" + str(sol.get_value(couples[i])) + "),"
    res = res[:-1] + "}" + " est un mariage stable."

    print(res)

In [226]:
print_SM(sol)

L'ensemble M={(h_0,f_3),(h_1,f_0),(h_2,f_2),(h_3,f_1)} est un mariage stable.
